In [1]:
import pandas as pd
from os import chdir
from sklearn.ensemble import RandomForestClassifier

In [2]:
chdir("../Compile")
import Data_filter as DF
import Analyzier

In [33]:
TrainingData = pd.DataFrame.from_csv("../../Data/training_data.csv")

In [34]:
Data = TrainingData['sentence'].apply(DF.ParseSentence)
Data = Data.to_frame()

In [35]:
Data['?'] = Data['sentence'].apply(lambda x: x[1]['?'])
Data['!'] = Data['sentence'].apply(lambda x: x[1]['!'])
Data['.'] = Data['sentence'].apply(lambda x: x[1]['.'])
Data['sum'] = Data['sentence'].apply(lambda x: Analyzier.AnalyzeWords(x[0]))
del(Data['sentence'])

In [36]:
Data.head(3)

,?,!,.,sum
1,0,1,0,0.0
2,1,0,0,0.0
3,1,0,0,0.5


In [37]:
Training = TrainingData['label'].values

In [38]:
model =  RandomForestClassifier(n_estimators=1000)

In [39]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
Xtr,Xval,Ytr,Yval = train_test_split(Data,Training,test_size=0.3,random_state=128)

In [40]:
model.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [41]:
print(f1_score(Yval, model.predict(Xval), average='micro'))

0.789285714286


In [86]:
Input = open("../../Data/data.txt")
PredictionData = []
for line in Input.readlines():
    PredictionData.append(line)
del(PredictionData[0])
Input.close()

In [87]:
PredictionData = pd.DataFrame(PredictionData)
PredictionData.columns = ['sentence']

In [88]:
PredictionData['parsed'] = PredictionData['sentence'].apply(DF.ParseSentence)
del(PredictionData['sentence'])

In [89]:
PredictionData['?'] = PredictionData['parsed'].apply(lambda x: x[1]['?'])
PredictionData['!'] = PredictionData['parsed'].apply(lambda x: x[1]['!'])
PredictionData['.'] = PredictionData['parsed'].apply(lambda x: x[1]['.'])
PredictionData['sum'] = PredictionData['parsed'].apply(lambda x: Analyzier.AnalyzeWords(x[0]))
del(PredictionData['parsed'])

In [90]:
Answer = model.predict(PredictionData)

In [91]:
Output = open('result.txt', 'w')
Output.write("\"label\""+"\n")

8

In [92]:
for i in Answer:
    Output.write("\""+str(i)+"\""+"\n")
Output.close()